In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
df=pd.read_csv('https://drive.google.com/uc?export=download&id=17CYveamtmveRPLiYvLXYXPKAvVuQQ6Ba')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
df=df.drop(columns=["RowNumber", "CustomerId", "Surname"])
le=LabelEncoder()
df["Gender"]=le.fit_transform(df["Gender"])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [ ]:

df=pd.get_dummies(df, columns=["Geography"], drop_first=True)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,False,False
1,608,0,41,1,83807.86,1,0,1,112542.58,0,False,True
2,502,0,42,8,159660.80,3,1,0,113931.57,1,False,False
3,699,0,39,1,0.00,2,0,0,93826.63,0,False,False
4,850,0,43,2,125510.82,1,1,1,79084.10,0,False,True


In [ ]:
X=df.drop("Exited", axis=1).values
y=df["Exited"].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
class ANN(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [ ]:
input_size = X_train.shape[1]
model = ANN(input_size)
criterion = nn.BCELoss() # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Convert arrays to tensors
inputs = torch.tensor(X_train, dtype=torch.float32)
labels = torch.tensor(y_train, dtype=torch.float32)
epochs = 800
for epoch in range(epochs):

    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, labels.unsqueeze(1))

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1)%10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/800], Loss: 0.6593
Epoch [20/800], Loss: 0.6415
Epoch [30/800], Loss: 0.6231
Epoch [40/800], Loss: 0.6030
Epoch [50/800], Loss: 0.5807
Epoch [60/800], Loss: 0.5571
Epoch [70/800], Loss: 0.5337
Epoch [80/800], Loss: 0.5123
Epoch [90/800], Loss: 0.4945
Epoch [100/800], Loss: 0.4808
Epoch [110/800], Loss: 0.4705
Epoch [120/800], Loss: 0.4627
Epoch [130/800], Loss: 0.4564
Epoch [140/800], Loss: 0.4509
Epoch [150/800], Loss: 0.4460
Epoch [160/800], Loss: 0.4414
Epoch [170/800], Loss: 0.4370
Epoch [180/800], Loss: 0.4326
Epoch [190/800], Loss: 0.4283
Epoch [200/800], Loss: 0.4239
Epoch [210/800], Loss: 0.4195
Epoch [220/800], Loss: 0.4148
Epoch [230/800], Loss: 0.4101
Epoch [240/800], Loss: 0.4052
Epoch [250/800], Loss: 0.4002
Epoch [260/800], Loss: 0.3952
Epoch [270/800], Loss: 0.3903
Epoch [280/800], Loss: 0.3855
Epoch [290/800], Loss: 0.3808
Epoch [300/800], Loss: 0.3764
Epoch [310/800], Loss: 0.3723
Epoch [320/800], Loss: 0.3685
Epoch [330/800], Loss: 0.3651
Epoch [340/800], Lo

In [ ]:
with torch.no_grad():
    y_predicted = model(torch.tensor(X_test, dtype=torch.float32))
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)).sum()/float(y_test.shape[0])
    print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8590
